<a href="https://colab.research.google.com/github/amoyag/Biofisica/blob/main/S1_introduccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#@title Setup mínimo para Colab (NumPy, SciPy, Matplotlib, Pandas)
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use('seaborn-v0_8')


## Ecuaciones diferenciales para la simulación de circuitos biológicos.
En biología de sistemas, las concentraciones de mRNA/proteína cambian en el tiempo según producción y degradación/dilución. Un gen regulado simple se modela como:

$$\frac{dY}{dt} = P(t) - \alpha Y$$


donde $P(t)$ es la tasa efectiva de producción (que puede depender de entradas/reguladores) y $\alpha$ es la tasa de pérdida. Esta formulación permite estudiar tiempos de respuesta, estados estacionarios y robustez de los circuitos$^1$.

En el taller 3, discutimos que proteínas y enzimas pueden comportarse como elementos de computación; las ODEs capturan esa computación distribuida sin “cables”, mediada por difusiones y acoplamientos alostéricos.


[1. Ver capítulos 1 y 2 de Alon U. Introduction to systems Biology]

En estas prácticas vamos a usar una API mínima que puedas “montar” como LEGO:

```python
def model(t, y, params): — modelo ODE del circuito (devuelve la derivada de y respecto al tiempo).
```
```python
simulate(model, y0, t_span, t_eval, params) — integrador común (SciPy).
plot_timeseries(sol, labels) — gráfica unificada.
```
```python
Bloques de entrada (step_input, pulse_input) y funciones reguladoras (hill_activation, hill_repression)
```

Así podrás usar el código para realizar simulaciones de circuitos biológicos y entender sus propiedades con una compresión mínima de Python.

In [ ]:

#@title Bloques reutilizables: simulación y visualización

def simulate(model, y0, t_span, t_eval, params):
    """Integra un sistema ODE con solve_ivp."""
    sol = solve_ivp(lambda t, y: model(t, y, params),
                    t_span=t_span, y0=y0, t_eval=t_eval,
                    method='RK45', rtol=1e-6, atol=1e-9)
    return sol

def plot_timeseries(sol, labels=None, title="Dinámica temporal"):
    plt.figure(figsize=(7,4))
    for i in range(sol.y.shape[0]):
        lab = labels[i] if labels and i < len(labels) else f"var{i}"
        plt.plot(sol.t, sol.y[i], label=lab)
    plt.xlabel("Tiempo")
    plt.ylabel("Concentración (a.u.)")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Regulación tipo Hill (activación y represión)
def hill_activation(x, K, n):
    """f(x) = x^n / (K^n + x^n)"""
    return (x**n) / (K**n + x**n)

def hill_repression(x, K, n):
    """f_rep(x) = 1 / (1 + (x/K)^n)"""
    return 1.0 / (1.0 + (x / K)**n)

# Entradas (estímulos)
def step_input(t, t_on=10.0, val_before=0.0, val_after=1.0):
    return val_before if t < t_on else val_after

def pulse_input(t, t_on=10.0, t_off=20.0, val=1.0):
    return val if t_on <= t < t_off else 0.0


In [ ]:

#@title Ejemplo 1: decaimiento exponencial (ODE 1 variable)

def linear_decay_model(t, y, params):
    alpha = params.get("alpha", 0.5)
    dy = -alpha*y[0]
    return [dy]

y0 = [1.0]
t_eval = np.linspace(0, 20, 400)
sol = simulate(linear_decay_model, y0, (0, 20), t_eval, {"alpha":0.3})
plot_timeseries(sol, labels=["Y"], title="Decaimiento exponencial: dY/dt = -αY")


In [ ]:

#@title Ejemplo 2: crecimiento logístico (competencia/recursos)

def logistic_model(t, y, params):
    r = params.get("r", 1.0)
    K = params.get("K", 10.0)
    dy = r*y[0]*(1 - y[0]/K)
    return [dy]

y0 = [0.5]
t_eval = np.linspace(0, 30, 600)
sol = simulate(logistic_model, y0, (0, 30), t_eval, {"r":0.8, "K":10})
plot_timeseries(sol, labels=["Y"], title="Crecimiento logístico: dY/dt = rY(1-Y/K)")


## Lógica Booleana vs lógica difusa
La actividad de una proteína con dos entradas puede aproximarse por AND/OR (Boole) o por una respuesta gradual tipo Hill (difusa). Los sistemas vivos suelen operar “entre” ambos extremos, beneficiándose de respuestas continuas y robustas. Recuerda el [paper de Dennis Bray.](https://www.nature.com/articles/376307a0)

In [ ]:

#@title Comparación Boole vs. difusa

def boolean_output(a, b, mode="AND", thr=0.5):
  #thr es el umbral de valores A y B que dan respuesta
    A = 1.0 if a >= thr else 0.0
    B = 1.0 if b >= thr else 0.0
    if mode == "AND":
        return 1.0 if (A==1.0 and B==1.0) else 0.0
    elif mode == "OR":
        return 1.0 if (A==1.0 or B==1.0) else 0.0
    elif mode == "NAND":
        return 0.0 if (A==1.0 and B==1.0) else 1.0
    else:
        raise ValueError("mode desconocido")

def fuzzy_output(a, b, K=0.5, n=2):
    # salida proporcional a suma de activaciones tipo Hill
    return 0.5*(hill_activation(a, K, n) + hill_activation(b, K, n))

# Barrido de entradas y visualización
A_vals = np.linspace(0, 1, 51)
B_vals = np.linspace(0, 1, 51)

Z_bool = np.zeros((len(A_vals), len(B_vals)))
Z_fuzzy = np.zeros_like(Z_bool)

for i, A in enumerate(A_vals):
    for j, B in enumerate(B_vals):
        Z_bool[i,j] = boolean_output(A, B, mode="AND", thr=0.5)
        Z_fuzzy[i,j] = fuzzy_output(A, B, K=0.5, n=2)

fig, axs = plt.subplots(1,2, figsize=(9,4))
im0 = axs[0].imshow(Z_bool, origin='lower', extent=[0,1,0,1], vmin=0, vmax=1, cmap='viridis')
axs[0].set_title("Salida Boole (AND)")
axs[0].set_xlabel("Entrada B"); axs[0].set_ylabel("Entrada A")
plt.colorbar(im0, ax=axs[0])

im1 = axs[1].imshow(Z_fuzzy, origin='lower', extent=[0,1,0,1], vmin=0, vmax=1, cmap='viridis')
axs[1].set_title("Salida difusa (Hill sum)")
axs[1].set_xlabel("Entrada B"); axs[1].set_ylabel("Entrada A")
plt.colorbar(im1, ax=axs[1])
plt.tight_layout()
plt.show()


### Ejercicio
Juega con $n$ (coeficiente de Hill) y $K$ (umbral). ¿Cuándo la respuesta difusa se acerca a un interruptor digital? Relaciona esto con la idea de “proteína interruptor” vs. “regulación gradual” que [describe Bray.]((https://www.nature.com/articles/376307a0))

## Un circuito mínimo: gen activado por una entrada (ODE modular)
Vamos a simular el caso más simple de circuito, $X \rightarrow Y$.
Esta simulación introduce el patrón clave que reutilizaremos en sesiones posteriores.

La variación de la concentración de $Y$ en el tiempo depende de su producción y su tasa de eliminación. El término de producción viene dado por la función de Hill mientras que la tasa de eliminación se describe mediante el decaimiento exponencial que hemos visto antes. Así,

$$\frac{dY}{dt} = P(t) - \alpha Y$$



A continuación verás la simulación integrada en un bloque de código.

In [ ]:
#@title Circuito 0: gen Y activado por una entrada S(t) con degradación

def gene_simple_model(t, y, params):
    # y[0] := Y (proteína)
    S_func = params["S_func"]                 # entrada tiempo-dependiente (es una función)
    S_value = S_func(t)                       # Evaluar la función S_func en el tiempo t
    Pmax = params.get("Pmax", 1.0)          # producción máxima
    K = params.get("K", 0.5)
    n = params.get("n", 2)
    alpha = params.get("alpha", 0.4)
    prod = Pmax * hill_activation(S_value, K, n)  # producción regulada, ahora con el valor numérico de S
    dy = prod - alpha*y[0]                  # balance producción/decadencia
    return [dy]

y0 = [0.0]
t_eval = np.linspace(0, 60, 900)
params = {"S_func": lambda t: step_input(t, t_on=10, val_before=0.0, val_after=1.0),
          "Pmax": 1.0, "K":0.5, "n":2, "alpha":0.1}
sol = simulate(gene_simple_model, y0, (0, 60), t_eval, params)
plot_timeseries(sol, labels=["Y"], title="Entrada escalón → respuesta de Y (activación Hill)")

En esta simulación, al ser sencilla, es fácil identificar dónde están los parámetros que podemos manipular para estudiar el comportamiento del sistema. Vamos a reescribir esta simulación en módulos, para ilustrar cómo puedes montar una simulación combinando los módulos. Este es el esquema que seguiremos en la práctica.

In [ ]:

#@title Módulo 1 · Modelo ODE (solo dinámica)
def model_gene_activation(t, y, params):
    """
    y[0] := Y (proteína/salida)
    El modelo usa una entrada temporal S(t) y una regulación tipo Hill para la producción.
    """
    # --- Lectura de entrada temporal --- #

    S_func = params["inputs"]["S"] # Obtener la función de entrada
    S_t = S_func(t)                  # Evaluar la función en el tiempo t

    # --- Parámetros del modelo (regulación y pérdida) --- #
    Pmax  = params["model"].get("Pmax", 1.0)
    K_act = params["model"].get("K", 0.5)
    n_act = params["model"].get("n", 2)
    alpha = params["model"].get("alpha", 0.1)

    # --- Función de activación (inyectada como submódulo) --- #
    hill_activation = params["functions"]["hill_activation"]

    # --- Producción y balance --- #
    prod = Pmax * hill_activation(S_t, K_act, n_act)
    dy   = prod - alpha * y[0]
    return [dy]


In [ ]:

#@title Módulo 2 · Parámetros y entradas (config)
import numpy as np

# Submódulos reutilizables de lógica/entradas
def hill_activation(x, K, n):
    return (x**n) / (K**n + x**n)

def step_input(t, t_on=10.0, val_before=0.0, val_after=1.0):
    return val_before if t < t_on else val_after

def pulse_input(t, t_on=10.0, t_off=20.0, val=1.0):
    return val if t_on <= t < t_off else 0.0

# Constructor de configuración
def build_config(
    model_params=None,
    input_func=None,
    functions=None,
    y0=None,
    t_span=(0, 60),
    t_eval=None
):
    """
    Devuelve un diccionario 'params' con:
      - params['model']     : dict de parámetros del modelo ODE
      - params['inputs']    : dict con funciones de entrada temporal
      - params['functions'] : dict con funciones auxiliares (Hill, etc.)
      - y0, t_span, t_eval  : condiciones iniciales y malla temporal
    """
    # Defaults
    if model_params is None:
        model_params = {"Pmax": 1.0, "K": 0.5, "n": 2, "alpha": 0.1}

    if input_func is None:
        input_func = lambda t: step_input(t, t_on=10, val_before=0.0, val_after=1.0)

    if functions is None:
        functions = {"hill_activation": hill_activation}

    if y0 is None:
        y0 = [0.0]

    if t_eval is None:
        t_eval = np.linspace(t_span[0], t_span[1], 900)

    params = {
        "model": model_params,
        "inputs": {"S": input_func},
        "functions": functions,
        "meta": {"name": "Circuito 0 · Activación simple"}
    }
    return params, y0, t_span, t_eval


# --- Entradas con ruido (ruido blanco gaussiano) ---


def noisy_input(t, base=1.0, sigma=0.2, clip_min=None, clip_max=None, rng=None):
    """
    Señal de entrada ruidosa: S(t) = base + N(0, sigma)
    - base: nivel medio de la señal
    - sigma: desviación estándar del ruido gaussiano
    - clip_min / clip_max: límites para recortar la señal (opcional)
    - rng: instancia de generador np.random.Generator para reproducibilidad (opcional)
    """
    # Generador aleatorio (permite reproducibilidad si se pasa rng)
    _rng = rng if rng is not None else np.random.default_rng()
    s = base + _rng.normal(loc=0.0, scale=sigma)

    # Recorte opcional para evitar valores negativos o excesivos
    if clip_min is not None:
        s = max(clip_min, s)
    if clip_max is not None:
        s = min(clip_max, s)
    return s


def noisy_pulse_input(t, t_on=10.0, t_off=30.0, base_on=1.0, base_off=0.0,
                      sigma_on=0.2, sigma_off=0.05, clip_min=0.0, clip_max=None, rng=None):
    """
    Pulso con ruido: antes y después del pulso hay un 'baseline' con su propio sigma;
    durante el pulso, otro 'baseline' y sigma.
    Útil para estudiar robustez cuando la señal solo está presente en un intervalo.
    """
    _rng = rng if rng is not None else np.random.default_rng()
    if t_on <= t < t_off:
        s = base_on + _rng.normal(0.0, sigma_on)
    else:
        s = base_off + _rng.normal(0.0, sigma_off)
    if clip_min is not None:
        s = max(clip_min, s)
    if clip_max is not None:
        s = min(clip_max, s)
    return s


def ramp_with_noise_input(t, t_start=10.0, slope=0.08, max_val=2.0,
                          sigma=0.1, clip_min=0.0, clip_max=None, rng=None):
    """
    Rampa con ruido: S(t) = ramp(t) + N(0, sigma)
    - Útil para mostrar filtrado temporal: el circuito atenúa el ruido de alta frecuencia.
    """
    _rng = rng if rng is not None else np.random.default_rng()
    val = slope * max(0.0, (t - t_start))
    val = min(val, max_val)
    s = val + _rng.normal(0.0, sigma)
    if clip_min is not None:
        s = max(clip_min, s)
    if clip_max is not None:
        s = min(clip_max, s)
    return s


In [ ]:

#@title Módulo 3 · Ejecución (integración numérica)
from scipy.integrate import solve_ivp

def run_simulation(model_func, params, y0, t_span, t_eval,
                   method="RK45", rtol=1e-6, atol=1e-9):
    """
    Integra el sistema ODE usando solve_ivp.
    """
    sol = solve_ivp(lambda t, y: model_func(t, y, params),
                    t_span=t_span, y0=y0, t_eval=t_eval,
                    method=method, rtol=rtol, atol=atol)
    return sol


In [ ]:

#@title Módulo 4 · Visualización (y export opcional)
import matplotlib.pyplot as plt
import pandas as pd

def plot_timeseries(sol, labels=None, title=None):
    plt.figure(figsize=(7,4))
    for i in range(sol.y.shape[0]):
        lab = labels[i] if labels and i < len(labels) else f"var{i}"
        plt.plot(sol.t, sol.y[i], label=lab)
    plt.xlabel("Tiempo")
    plt.ylabel("Concentración (a.u.)")
    plt.title(title or "Dinámica temporal")
    plt.legend()
    plt.tight_layout()
    plt.show()

def to_dataframe(sol, labels=None):
    data = {"t": sol.t}
    for i in range(sol.y.shape[0]):
        key = labels[i] if labels and i < len(labels) else f"var{i}"
        data[key] = sol.y[i]
    return pd.DataFrame(data)

def save_csv(df, path="resultados.csv"):
    df.to_csv(path, index=False)
    return path


In [ ]:

#@title Módulo extra: análisis de ruido en fase estacionaria
def std_steady(sol, window=100):
    """Calcula la desviación estándar de la salida en los últimos 'window' puntos."""
    return np.std(sol.y[0][-window:])

def analyze_noise_effect(model_func, base_params, y0, t_span, t_eval,
                         noise_sigma_list, n_list):
    """
    Ejecuta simulaciones para distintos niveles de ruido y cooperatividad.
    Devuelve un diccionario con std en fase estacionaria.
    """
    results = {}
    for n in n_list:
        results[n] = {}
        for sigma in noise_sigma_list:
            # Configuración con ruido
            params = {
                "model": {**base_params, "n": n},
                "inputs": {"S": lambda t: noisy_input(t, base=1.0, sigma=sigma)},
                "functions": {"hill_activation": hill_activation},
                "meta": {"name": f"n={n}, sigma={sigma}"}
            }
            sol = run_simulation(model_func, params, y0, t_span, t_eval)
            results[n][sigma] = std_steady(sol)
    return results


### Cómo preparar tu simulación
A continuación tienes un ejemplo sobre cómo preparar tu simulación combinando los módulos que hemos definido. Esto es en esencia lo que debes hacer en tu práctica.

In [ ]:

#@title Preparar la simulación: elegir modelo + config + ejecutar + visualizar


# 1) Elegir el modelo (Módulo 1)
model_func = model_gene_activation

# 2) Construir configuración (Módulo 2)
#    Cambia input_func para step/pulse y model_params para sensibilidad
params, y0, t_span, t_eval = build_config(
    model_params={"Pmax": 1.2, "K": 0.4, "n": 3, "alpha": 0.12},
    input_func=lambda t: step_input(t, t_on=8, val_before=0.0, val_after=1.0)
    #input_func=lambda t: pulse_input(t, t_on=3, t_off=40, val=01.0, )
    #input_func=lambda t: step_input(t, t_on=45) + pulse_input(t, t_on=30, t_off=40)

    )

# 3) Ejecutar (Módulo 3)
sol = run_simulation(model_func, params, y0, t_span, t_eval)

# 4) Visualizar y exportar (Módulo 4)
plot_timeseries(sol, labels=["Y"], title=params["meta"]["name"])
#df = to_dataframe(sol, labels=["Y"])
#_ = save_csv(df, path="circuito0_activacion.csv")
#df.head()


Vamos a ver paso a paso cómo hemos preparado la simulación.



#### 1. Elegir el modelo. El modelo define las ecuaciones diferenciales que describen el circuito.
    ```python
    model_func = model_gene_activation
    ```
    Este modelo representa u gen activado por una señal externa (factor de transcripción).

#### 2. Configurar parámetros y entradas. Aquí defines:

  - Parámetros del modelo (p.ej., Pmax, K, n, alpha).
  - Entrada temporal (la señal que activa el circuito).
  - Funciones auxiliares (como la activación tipo Hill).
  Puedes usar funciones predefinidas `step_input, pulse_input`, etc.) o crear tu propia entrada con `lambda`, una forma rápida de personalizar la entrada sin escribir una función completa. Por ejemplo, puedes cambiar el tiempo de activación (t_on) o el valor del estímulo.
  ```python
  params, y0, t_span, t_eval = build_config(
      model_params={"Pmax": 1.2, "K": 0.4, "n": 3, "alpha": 0.12},
      input_func=lambda t: step_input(t, t_on=8, val_before=0.0, val_after=1.0))
  ```
Jugando con la función de input, puedes ver qué pasa si la activación dura poco o si funciona en pulsos.
```python
params, y0, t_span, t_eval = build_config(
    model_params={"Pmax": 1.2, "K": 0.4, "n": 3, "alpha": 0.12},
    #input_func=lambda t: step_input(t, t_on=8, val_before=0.0, val_after=1.0)
    input_func=lambda t: pulse_input(t, t_on=3, t_off=40, val=01.0)
    )
```
Incluso puedes combinar varias entradas:
```python
input_func=lambda t: step_input(t, t_on=10) + pulse_input(t, t_on=30, t_off=40)
```


#### 3. Ejecutar la simulación
Usa el integrador para resolver las ODEs.
```python
sol = run_simulation(model_func, params, y0, t_span, t_eval)
```

#### 4. Visualizar y analizar
Puedes ver la dinámica y guardar los resultados
```python
plot_timeseries(sol, labels=["Y"], title=params["meta"]["name"])
df = to_dataframe(sol, labels=["Y"])
```

## **Ejercicios: Propiedades de la regulación simple**

Ahora vas a explorar cómo cambian las propiedades dinámicas y funcionales de un circuito simple (gen activado por una señal) según sus parámetros. Estos ejercicios conectan con los conceptos de **tiempo de respuesta**, **sensibilidad**, **robustez** y **saturación** que describe Uri Alon en Introduction to Systems Biology.

***

#### **Ejercicio 1: Tiempo de respuesta**

**Objetivo:** Ver cómo la tasa de degradación ($\alpha$) afecta la rapidez con que el sistema alcanza el estado estacionario.

1.  Simula el circuito con $\alpha$ = 0.05, 0.1, 0.2.
2.  Calcula el tiempo en que la salida alcanza el **50% del valor estacionario** ($t_{\frac{1}{2}}$).
3.  **Pregunta:** ¿Por qué el tiempo de respuesta disminuye al aumentar $\alpha$?

**Código base:**

```python
def response_time(sol, frac=0.5):
    steady = sol.y[0][-1]
    target = frac * steady
    idx = np.where(sol.y[0] >= target)[0][0]
    return sol.t[idx]

for alpha in [0.05, 0.1, 0.2]:
    params, y0, t_span, t_eval = build_config(
        model_params={"Pmax": 1.0, "K": 0.5, "n": 2, "alpha": alpha},
        input_func=lambda t: step_input(t, t_on=10)
    )
    sol = run_simulation(model_gene_activation, params, y0, t_span, t_eval)
    t_half = response_time(sol)
    print(f"alpha={alpha}, t_1/2={t_half:.2f}")
    plot_timeseries(sol, labels=["Y"], title=f"alpha={alpha}")
```

***

#### **Ejercicio 2: Sensibilidad y cooperatividad**

**Objetivo:** Explorar cómo el coeficiente de Hill ($n$) cambia la forma de la respuesta.

1. Observa las curvas de activación para $n = 1, 4, 10$.
2. Explica por qué la cooperatividad alta produce una respuesta más “digital” cuando la entrada varía gradualmente.
3.  **Pregunta:**  ¿Qué ventajas tiene esto para circuitos que deben tomar decisiones claras?


**Código base:**

```python
def hill_activation(x, K, n):
    return (x**n) / (K**n + x**n)

S_vals = np.linspace(0, 2.0, 400)  # barrido de la señal de entrada
K = 0.5

plt.figure(figsize=(7,4))
for n in [1, 4, 10]:
    act = hill_activation(S_vals, K=K, n=n)
    plt.plot(S_vals, act, label=f"n={n}")
plt.axvline(K, color='gray', linestyle='--', alpha=0.5, label="K (umbral)")
plt.xlabel("Entrada S")
plt.ylabel("Producción normalizada f(S)")
plt.title("Curvas de activación tipo Hill (efecto de n)")
plt.legend()
plt.tight_layout()
plt.show()
```


***

#### **Ejercicio 3: Umbral de activación**

**Objetivo:** Ver cómo el parámetro $K$ afecta la sensibilidad.

1.  Observa las curvas de activación para $ K = 0.2, 0.4, 1$.
2.  **Pregunta:** ¿Cómo cambia el punto donde la salida empieza a crecer?

**Código base:**

```python
# Curvas de activación para distintos K
S_vals = np.linspace(0, 2, 400)
n = 2  # mantenemos n fijo
plt.figure(figsize=(7,4))
for K in [0.1, 0.4, 1.0]:
    act = [hill_activation(S, K=K, n=n) for S in S_vals]
    plt.plot(S_vals, act, label=f"K={K}")
plt.xlabel("Entrada S")
plt.ylabel("Producción normalizada f(S)")
plt.title("Efecto de K en la curva de activación")
plt.legend()
plt.show()
```

***


#### **Ejercicio 4: Robustez ante ruido**

Usa el módulo `analyze_noise_effect` para calcular la desviación estándar de la salida en la fase estacionaria para:

- $n=1$ y $n=4$
- $\sigma = 0.0, 0.2, 0.5$

Dado que vas a calcular valores de desviación estándar en lugar de hacer una representación gráfica, debes usar el módulo extra de análisis numérico.

Compara los resultados y responde:

¿Cómo cambia la variabilidad de la salida cuando aumenta el ruido en la entrada?
¿Por qué el circuito no transmite todo el ruido?
¿Qué papel juega la cooperatividad ($n$)?



Pista: El circuito actúa como un filtro temporal: atenúa fluctuaciones rápidas.


**Código base:**

```python
# Este código base ejecuta la simulación para varios valores de sigma y n

# Configuración común
t_span = (0, 60)
t_eval = np.linspace(0, 60, 600)
y0 = [0.0]
base_params = {"Pmax": 1.0, "K": 0.5, "alpha": 0.05}

# Listas de valores para el análisis
noise_sigma_list = [0.0, 0.2, 0.5]
n_list = [1, 4]

# Ejecutar análisis (usa analyze_noise_effect)
results = analyze_noise_effect(model_gene_activation, base_params, y0,
                                t_span, t_eval, noise_sigma_list, n_list)

# Mostrar resultados
print("Comparación entrada vs salida (desviación estándar):")
for n in results:
    for sigma in results[n]:
        std_in, std_out = results[n][sigma]
        print(f"n={n}, sigma={sigma}: entrada={std_in:.4f}, salida={std_out:.4f}")


```


A continuación tienes un ejemplo de cómo quedaría la simulación que deberías preparar para hacer este ejercicio.

```python
#@title Simulción modular completa con ruido


# --- Módulo 1: Modelo ---
model_func = model_gene_activation  # definido previamente

# --- Módulo 2: Entradas y parámetros ---
# Elegimos noisy_input() puesto que vamos a hacer una simulación en la que hay ruido en la señal

def noisy_input(t, base=1.0, sigma=0.3, clip_min=None, clip_max=None, rng=None):
    _rng = rng if rng is not None else np.random.default_rng()
    s = base + _rng.normal(0.0, sigma)
    if clip_min is not None:
        s = max(clip_min, s)
    if clip_max is not None:
        s = min(clip_max, s)
    return s

# --- Módulo extra: análisis numérico ---
def std_steady(sol, window=100):
    """Desviación estándar de la salida en los últimos 'window' puntos."""
    return np.std(sol.y[0][-window:])

def std_input(params, t_eval, window=100):
    """Desviación estándar de la entrada en los últimos 'window' puntos."""
    input_vals = [params["inputs"]["S"](t) for t in t_eval[-window:]]
    return np.std(input_vals)

def analyze_noise_effect(model_func, base_params, y0, t_span, t_eval,
                         noise_sigma_list, n_list):
    """
    Ejecuta simulaciones para distintos niveles de ruido y cooperatividad.
    Devuelve un diccionario con std entrada y salida.
    """
    results = {}
    for n in n_list:
        results[n] = {}
        for sigma in noise_sigma_list:
            params = {
                "model": {**base_params, "n": n},
                "inputs": {"S": lambda t: noisy_input(t, base=1.0, sigma=sigma)},
                "functions": {"hill_activation": hill_activation},
                "meta": {"name": f"n={n}, sigma={sigma}"}
            }
            sol = run_simulation(model_func, params, y0, t_span, t_eval)
            std_out = std_steady(sol)
            std_in = std_input(params, t_eval)
            results[n][sigma] = (std_in, std_out)
    return results

# --- Configuración común ---
t_span = (0, 60)
t_eval = np.linspace(0, 60, 600)
y0 = [0.0]
base_params = {"Pmax": 1.0, "K": 0.5, "alpha": 0.05}

# --- Listas de valores para el análisis ---
noise_sigma_list = [0.0, 0.2, 0.5]
n_list = [1, 4]

# --- Ejecutar análisis ---
results = analyze_noise_effect(model_func, base_params, y0,
                                t_span, t_eval, noise_sigma_list, n_list)

# --- Mostrar resultados ---
print("Comparación entrada vs salida (desviación estándar):")
for n in results:
    for sigma in results[n]:
        std_in, std_out = results[n][sigma]
        print(f"n={n}, sigma={sigma}: entrada={std_in:.4f}, salida={std_out:.4f}")
```


***

#### **Ejercicio 5: Comparación lógica Booleana vs difusa**

**Objetivo:** Contrastar dos modos de integración de señales.

1.  Usa el bloque de lógica Booleana y el bloque difuso para la misma entrada doble.
2.  **Pregunta:** ¿Por qué la lógica difusa es más común en biología?



***


